In [1]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging)
import pandas as pd
from ema_problem_definitions import ema_problem

In [2]:
# enable logging
ema_logging.LOG_FORMAT = '%(message)s'
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [3]:
# import problem definition
model = ema_problem(1)

In [4]:
with MultiprocessingEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=11)

pool started
performing 11 scenarios * 1 policies * 1 model(s) = 11 experiments
1 cases completed
2 cases completed
3 cases completed
4 cases completed
5 cases completed
6 cases completed
7 cases completed
8 cases completed
9 cases completed
10 cases completed
11 cases completed
experiments finished
terminating pool


In [5]:
experiments

,VTG_percentage,average_driving_speed,offset_dep,offset_dwell,p_pref,p_smart,pref_home,pref_strictness,weekend_week_ratio,scenario,policy,model
0,0.673298,13.0,1.0,4.0,0.601951,0.765302,0.946319,0.356444,0.933842,0,None,EtmEVsModel
1,0.898279,6.0,4.0,2.0,0.258270,0.675922,0.569659,0.872238,0.437633,1,None,EtmEVsModel
2,0.964939,8.0,2.0,3.0,0.500356,0.558225,0.893629,0.092886,0.849961,2,None,EtmEVsModel
3,0.346791,20.0,0.0,2.0,0.893720,0.948392,0.348555,0.191447,0.455457,3,None,EtmEVsModel
4,0.564364,22.0,0.0,1.0,0.124503,0.253764,0.110787,0.637919,0.166448,4,None,EtmEVsModel
5,0.438879,19.0,2.0,0.0,0.305211,0.057125,0.439047,0.041703,0.799354,5,None,EtmEVsModel
6,0.070420,25.0,3.0,0.0,0.915050,0.094181,0.493325,0.592370,0.726552,6,None,EtmEVsModel
7,0.200146,15.0,4.0,3.0,0.392086,0.538553,0.792068,0.940462,0.073779,7,None,EtmEVsModel
8,0.159402,11.0,1.0,4.0,0.081457,0.299268,0.658714,0.534772,0.297691,8,None,EtmEVsModel
9,0.742090,16.0,3.0,0.0,0.783710,0.839559,0.221939,0.401298,0.603743,9,None,EtmEVsModel


In [6]:
pd.DataFrame(outcomes)

,min_average_battery_percentage,mean_average_battery_percentage,min_power_demand,mean_power_demand,max_power_demand,min_VTG_capacity,mean_VTG_capacity,max_VTG_capacity,min_mean_charging,mean_mean_charging,max_mean_charging
0,85.549399,90.847966,82.283792,4864.458612,24794.459004,82.456027,59473.634585,181642.222218,0.000575,0.029847,0.142759
1,83.810114,87.009608,6.538522,2020.343939,14702.343299,69.414199,20732.368813,67912.111438,0.000057,0.012289,0.084655
2,89.318490,92.925565,22.547388,5582.236454,39143.824101,66.538137,56492.371455,159204.099342,0.000115,0.033863,0.224368
3,81.618848,89.993894,62.955487,5613.333292,20766.046627,113.481656,78245.266552,242453.192061,0.000345,0.034641,0.124483
4,85.590857,88.209078,12.486359,3647.627702,16582.535325,21.697826,44219.589406,93378.887303,0.000115,0.022292,0.095517
5,92.759944,96.239853,46.819714,6955.087445,50853.605461,90.635293,67735.409444,141394.995985,0.000345,0.041999,0.293333
6,95.638369,97.687881,138.253651,7710.086898,143326.504696,22.986173,37296.209257,65422.497246,0.000920,0.045933,0.825977
7,84.226488,87.646728,12.780934,2797.278522,31118.183711,32.546075,23912.777945,62356.491637,0.000057,0.016982,0.179138
8,87.715070,89.685745,206.700650,3527.345235,10312.373246,30.026482,32446.964285,63947.426013,0.001207,0.021692,0.062471
9,84.105708,90.598079,15.954301,5022.983457,21547.737366,85.738358,67815.372151,214691.533532,0.000115,0.030843,0.125230
